**Part 1.** This notebook contains method for exploatory data analysis (EDA)

The aim of this notebook is understanding the data



In [ ]:
# from google.colab import drive
# import pandas as pd
# drive.mount('/content/drive')

In [ ]:
!pip install prophet -q

In [ ]:
import numpy as np 
import pandas as pd

import datetime

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import statsmodels.api as sm
# варианты:
# data = sm.datasets.longley.load_pandas()

data = sm.datasets.sunspots.load_pandas()
# data = sm.datasets.nile.load_pandas()
#data = sm.datasets.macrodata.load_pandas()
# data = sm.datasets.interest_inflation.load_pandas()
#data = sm.datasets.elnino.load_pandas()
# data = sm.datasets.co2.load_pandas()

In [ ]:
df = data.data
df.tail()

,YEAR,SUNACTIVITY
304,2004.0,40.4
305,2005.0,29.8
306,2006.0,15.2
307,2007.0,7.5
308,2008.0,2.9


In [ ]:
df['YEAR'] = df['YEAR'].astype('int')
df['DATE'] = df['YEAR'].map(lambda x: datetime.date(x, 1, 1))
df.tail()

,YEAR,SUNACTIVITY,DATE
304,2004,40.4,2004-01-01
305,2005,29.8,2005-01-01
306,2006,15.2,2006-01-01
307,2007,7.5,2007-01-01
308,2008,2.9,2008-01-01


In [ ]:
import plotly.express as px
fig = px.line(df, x="YEAR", y="SUNACTIVITY", title='SUNACTIVITY')
fig.show()

In [ ]:
df = df[['DATE', 'SUNACTIVITY']]
df.columns = ['ds', 'y']

In [ ]:
def get_stat(df, df_name):
  print(f'__статистика по  датафрейму {df_name}___')
  print('shape = ', df.shape)
  print('min_dt = ', df['ds'].min())
  print('max_dt = ', df['ds'].max())
  print('columns = ', list(df.columns))

In [ ]:
get_stat(df, 'df')

__статистика по  датафрейму df___
shape =  (309, 2)
min_dt =  1700-01-01
max_dt =  2008-01-01
columns =  ['ds', 'y']


In [ ]:
# разделяем на тренировочную и тестовую выборку, в тестовой выборке будет 30 последних наблюдений по времени
# так как prophet не может работать с данными за большое количество лет, мы снизу исключаем первые 50 лет. 
prophet_train_df = df[50:-30]
prophet_test_df = df[-30:]
# посмотрим на статистику по датасетам:
get_stat(prophet_train_df, 'prophet_train_df')
get_stat(prophet_train_df, 'prophet_train_df')

__статистика по  датафрейму prophet_train_df___
shape =  (229, 2)
min_dt =  1750-01-01
max_dt =  1978-01-01
columns =  ['ds', 'y']
__статистика по  датафрейму prophet_train_df___
shape =  (229, 2)
min_dt =  1750-01-01
max_dt =  1978-01-01
columns =  ['ds', 'y']


In [ ]:
# создаем модель prophet
from prophet import Prophet
model_prophet = Prophet(seasonality_mode='additive',
                  yearly_seasonality = False,
                  weekly_seasonality = False,
                  daily_seasonality = False,
                  ).add_seasonality(name='custom_seasonality',
                                    period=100,
                                    fourier_order = 100)
model_prophet.fit(prophet_train_df)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpmcwhbn54/rgtmuhk1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmcwhbn54/1s5p_lwc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=51629', 'data', 'file=/tmp/tmpmcwhbn54/rgtmuhk1.json', 'init=/tmp/tmpmcwhbn54/1s5p_lwc.json', 'output', 'file=/tmp/tmpmcwhbn54/prophet_modelogyrjvv6/prophet_model-20230206184002.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:40:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:40:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
forecast_train_df = model_prophet.predict(prophet_train_df)
forecast_test_df = model_prophet.predict(prophet_test_df)

In [ ]:
get_stat(forecast_train_df, 'forecast_train_df')
get_stat(forecast_test_df, 'forecast_test_df')

__статистика по  датафрейму forecast_train_df___
shape =  (229, 16)
min_dt =  1750-01-01 00:00:00
max_dt =  1978-01-01 00:00:00
columns =  ['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms', 'additive_terms_lower', 'additive_terms_upper', 'custom_seasonality', 'custom_seasonality_lower', 'custom_seasonality_upper', 'multiplicative_terms', 'multiplicative_terms_lower', 'multiplicative_terms_upper', 'yhat']
__статистика по  датафрейму forecast_test_df___
shape =  (30, 16)
min_dt =  1979-01-01 00:00:00
max_dt =  2008-01-01 00:00:00
columns =  ['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms', 'additive_terms_lower', 'additive_terms_upper', 'custom_seasonality', 'custom_seasonality_lower', 'custom_seasonality_upper', 'multiplicative_terms', 'multiplicative_terms_lower', 'multiplicative_terms_upper', 'yhat']


In [ ]:
# создадим датафрейм с будущими значениями, для данного датафрейма нет фактических y, будут только прогнозные значения
from datetime import timedelta
# (forecast_test_df['ds'].max() + timedelta(days=366)).strftime('%Y-%m-%d')
prophet_future_df = pd.DataFrame(columns=['ds', ])
prophet_future_df['ds'] = pd.date_range(start=(forecast_test_df['ds'].max() + timedelta(days=366)).strftime('%Y-%m-%d'), 
                                        periods=15,
                                        freq='YS',
                                        normalize=False)
get_stat(prophet_future_df, 'prophet_future_df')

__статистика по  датафрейму prophet_future_df___
shape =  (15, 1)
min_dt =  2009-01-01 00:00:00
max_dt =  2023-01-01 00:00:00
columns =  ['ds']


In [ ]:
# строим прогноз для датафрейма с будущими значениями
forecast_future_df = model_prophet.predict(prophet_future_df)
get_stat(forecast_future_df, 'forecast_future_df')

__статистика по  датафрейму forecast_future_df___
shape =  (15, 16)
min_dt =  2009-01-01 00:00:00
max_dt =  2023-01-01 00:00:00
columns =  ['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms', 'additive_terms_lower', 'additive_terms_upper', 'custom_seasonality', 'custom_seasonality_lower', 'custom_seasonality_upper', 'multiplicative_terms', 'multiplicative_terms_lower', 'multiplicative_terms_upper', 'yhat']


In [ ]:
# Считаем ошибки
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error as MSE


print('MAPE_test:', MAPE(prophet_test_df['y'], forecast_test_df['yhat']))
print('RMSE_test:', (MSE(prophet_test_df['y'], forecast_test_df['yhat']))**.5)
print('MSE_test:', MSE(prophet_test_df['y'], forecast_test_df['yhat']))



print('MAPE_train:', MAPE(prophet_train_df[prophet_train_df['ds']>datetime.date(1749,1,1)].iloc[:].set_index('ds')['y'], 
                          forecast_train_df.iloc[:].set_index('ds')['yhat']
                          )
      )
print('RMSE_train:', (MSE(prophet_train_df[prophet_train_df['ds']>datetime.date(1749,1,1)]['y'], forecast_train_df['yhat']))**.5)
print('MSE_train:', MSE(prophet_train_df[prophet_train_df['ds']>datetime.date(1749,1,1)]['y'], forecast_train_df['yhat']))

MAPE_test: 1.082534419768037
RMSE_test: 41.427096292883114
MSE_test: 1716.20430725981
MAPE_train: 989944651496877.0
RMSE_train: 25.3805568672275
MSE_train: 644.1726668905691


In [ ]:
# объединяем прогнозы для отрисовки 
columns = ['ds', 'trend', 'custom_seasonality', 'yhat']
df_pred = pd.concat([forecast_train_df, forecast_test_df, forecast_future_df])[columns]
get_stat(df_pred, 'финальный датасет со всеми прогнозами')
df_pred.tail()

__статистика по  датафрейму финальный датасет со всеми прогнозами___
shape =  (274, 4)
min_dt =  1750-01-01 00:00:00
max_dt =  2023-01-01 00:00:00
columns =  ['ds', 'trend', 'custom_seasonality', 'yhat']


,ds,trend,custom_seasonality,yhat
10,2019-01-01,87.574409,-63.374664,24.199745
11,2020-01-01,87.680079,-62.692893,24.987186
12,2021-01-01,87.786039,-46.384610,41.401429
13,2022-01-01,87.891709,-24.535229,63.356480
14,2023-01-01,87.997380,23.201012,111.198392


In [ ]:
# проверяем, что предудущий шаг отработал верно
forecast_train_df.shape[0]+forecast_test_df.shape[0]+forecast_future_df.shape[0] == df_pred.shape[0]

True

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'],
                    mode='lines+markers',
                    name='y'))
fig.add_trace(go.Scatter(x=df_pred['ds'], y=df_pred['yhat'],
                    mode='lines+markers',
                    name='yhat'))
fig.add_trace(go.Scatter(x=df_pred['ds'], y=df_pred['custom_seasonality'],
                    mode='lines',
                    name='custom_seasonality'))
fig.add_trace(go.Scatter(x=df_pred['ds'], y=df_pred['trend'],
                    mode='lines',
                    name='trend'))

fig.show()